In [ ]:
!wget v2_Annotations_Train_mscoco.zip "https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Annotations_Train_mscoco.zip"
!wget v2_Annotations_Val_mscoco.zip "https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Annotations_Val_mscoco.zip"
!wget v2_Questions_Train_mscoco.zip "https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Questions_Train_mscoco.zip"
!wget v2_Questions_Val_mscoco.zip "https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Questions_Val_mscoco.zip"
!wget v2_Questions_Test_mscoco.zip "https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Questions_Test_mscoco.zip"

# VQA Input Images (COCO)
!wget train2014.zip "http://images.cocodataset.org/zips/train2014.zip"
!wget val2014.zip "http://images.cocodataset.org/zips/val2014.zip"
!wget test2015.zip "http://images.cocodataset.org/zips/test2015.zip"

--2023-10-02 17:59:03--  http://v2_annotations_train_mscoco.zip/
Resolving v2_annotations_train_mscoco.zip (v2_annotations_train_mscoco.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘v2_annotations_train_mscoco.zip’
--2023-10-02 17:59:03--  https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Annotations_Train_mscoco.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.195.64, 52.217.204.64, 54.231.225.16, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.195.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21708861 (21M) [application/zip]
Saving to: ‘v2_Annotations_Train_mscoco.zip’

v2_Annotations_Trai 100%[===================>]  20.70M  25.7MB/s    in 0.8s    

2023-10-02 17:59:04 (25.7 MB/s) - ‘v2_Annotations_Train_mscoco.zip’ saved [21708861/21708861]

FINISHED --2023-10-02 17:59:04--
Total wall clock time: 1.1s
Downloaded: 1 files, 21M in 0.8s (25.7 MB/s)
--2023-10-02 17:59:04--  http://v2_annotations_val

In [ ]:
!unzip /content/v2_Annotations_Train_mscoco.zip
!unzip /content/v2_Annotations_Val_mscoco.zip
!unzip /content/v2_Questions_Test_mscoco.zip
!unzip /content/v2_Questions_Train_mscoco.zip
!unzip /content/v2_Questions_Val_mscoco.zip

!rm /content/v2_Annotations_Train_mscoco.zip
!rm /content/v2_Annotations_Val_mscoco.zip
!rm /content/v2_Questions_Test_mscoco.zip
!rm /content/v2_Questions_Train_mscoco.zip
!rm /content/v2_Questions_Val_mscoco.zip

!unzip /content/val2014.zip
!rm /content/val2014.zip

!unzip /content/train2014.zip
!rm /content/train2014.zip

!unzip /content/test2015.zip
!rm /content/test2015.zip

Streaming output truncated to the last 5000 lines.
 extracting: test2015/COCO_test2015_000000303195.jpg  
 extracting: test2015/COCO_test2015_000000481207.jpg  
 extracting: test2015/COCO_test2015_000000229167.jpg  
 extracting: test2015/COCO_test2015_000000466120.jpg  
 extracting: test2015/COCO_test2015_000000237368.jpg  
 extracting: test2015/COCO_test2015_000000009689.jpg  
 extracting: test2015/COCO_test2015_000000064513.jpg  
 extracting: test2015/COCO_test2015_000000058293.jpg  
 extracting: test2015/COCO_test2015_000000449819.jpg  
 extracting: test2015/COCO_test2015_000000106852.jpg  
 extracting: test2015/COCO_test2015_000000227623.jpg  
 extracting: test2015/COCO_test2015_000000250613.jpg  
 extracting: test2015/COCO_test2015_000000347395.jpg  
 extracting: test2015/COCO_test2015_000000388934.jpg  
 extracting: test2015/COCO_test2015_000000047455.jpg  
 extracting: test2015/COCO_test2015_000000523288.jpg  
 extracting: test2015/COCO_test2015_000000368687.jpg  
 extracting: t

In [ ]:
import numpy as np
import tensorflow as tf
#from tensorflow import keras
import random
from tensorflow.keras import layers
import keras
import pandas as pd
import tensorflow_datasets as tfds
import cv2 as cv

In [ ]:
import json


def read_json(file):

    with open(file) as f:
        data = json.load(f)
    return data

ann_train = read_json('/content/v2_mscoco_train2014_annotations.json')
ann_val = read_json('/content/v2_mscoco_val2014_annotations.json')


qu_train = read_json('/content/v2_OpenEnded_mscoco_train2014_questions.json')
qu_test = read_json('/content/v2_OpenEnded_mscoco_test2015_questions.json')
qu_test_dev=read_json('/content/v2_OpenEnded_mscoco_test-dev2015_questions.json')
qu_val = read_json('/content/v2_OpenEnded_mscoco_val2014_questions.json')

In [ ]:
import pandas as pd

ann_train_df = pd.DataFrame(ann_train['annotations'])

In [ ]:
ann_val_df = pd.DataFrame(ann_val['annotations'])

In [ ]:
qu_train_df = pd.DataFrame(qu_train['questions'])

In [ ]:
qu_val_df = pd.DataFrame(qu_val['questions'])

In [ ]:
qu_test_df = pd.DataFrame(qu_test['questions'])
qu_test_df_dev = pd.DataFrame(qu_test_dev['questions'])

In [ ]:
qu_test_df=pd.concat((qu_test_df,qu_test_df_dev))

In [ ]:
inner_train_df = pd.merge(ann_train_df, qu_train_df, on = ['question_id', 'image_id'], how = 'inner')

In [ ]:
inner_val_df = pd.merge(ann_val_df, qu_val_df, on = ['question_id', 'image_id'], how = 'inner')

In [ ]:
qu_test_df.to_csv('QuTest_df.csv', index=False, encoding='utf-8')
inner_train_df.to_csv('Train_df.csv', index=False, encoding='utf-8')
inner_val_df.to_csv('Val_df.csv', index=False, encoding='utf-8')

In [ ]:
import pandas as pd
import glob

In [ ]:
#glob.glob("/content/train2014/*.jpg")
'/content/train2014/COCO_train2014_000000310514.jpg'.split("/")[-1]

'COCO_train2014_000000310514.jpg'

In [ ]:
train_img=pd.DataFrame({'path':glob.glob("/content/train2014/*.jpg")})
train_img['image_id']=train_img.path.str.split("/").map(
    lambda x: list(x)[-1]).map(
        lambda x: x.strip('0')).str.split("_").map(
            lambda x: list(x)[-1]).str.split(".").map(
            lambda x: list(x)[0]).str.strip("0")

In [ ]:
train_img.dtypes

path        object
image_id    object
dtype: object

In [ ]:
train_img.to_csv("train_imgs.csv", index=False)

In [ ]:
test_img=pd.DataFrame({'path':glob.glob("/content/test2015/*.jpg")})
test_img['image_id']=test_img.path.str.split("/").map(
    lambda x: list(x)[-1]).map(
        lambda x: x.strip('0')).str.split("_").map(
            lambda x: list(x)[-1]).str.split(".").map(
            lambda x: list(x)[0]).str.strip("0")

In [ ]:
test_img.to_csv("test_imgs.csv", index=False)

In [ ]:
val_img=pd.DataFrame({'path':glob.glob("/content/val2014/*.jpg")})
val_img['image_id']=val_img.path.str.split("/").map(
    lambda x: list(x)[-1]).map(
        lambda x: x.strip('0')).str.split("_").map(
            lambda x: list(x)[-1]).str.split(".").map(
            lambda x: list(x)[0]).str.strip("0")
val_img

,path,image_id
0,/content/val2014/COCO_val2014_000000389969.jpg,389969
1,/content/val2014/COCO_val2014_000000316396.jpg,316396
2,/content/val2014/COCO_val2014_000000393089.jpg,393089
3,/content/val2014/COCO_val2014_000000412916.jpg,412916
4,/content/val2014/COCO_val2014_000000075405.jpg,75405
...,...,...
40499,/content/val2014/COCO_val2014_000000009729.jpg,9729
40500,/content/val2014/COCO_val2014_000000310052.jpg,310052
40501,/content/val2014/COCO_val2014_000000220808.jpg,220808
40502,/content/val2014/COCO_val2014_000000577619.jpg,577619


In [ ]:
val_img.to_csv("val_imgs.csv", index=False)

In [ ]:
QuTest_df = pd.read_csv('QuTest_df.csv')

In [ ]:
test_imgs = pd.read_csv('test_imgs.csv')

In [ ]:
Train_df = pd.read_csv('Train_df.csv')
train_imgs = pd.read_csv('train_imgs.csv')
Val_df = pd.read_csv('Val_df.csv')
val_imgs = pd.read_csv('val_imgs.csv')

In [ ]:
inner_Train_df = pd.merge(Train_df, train_imgs, on = ['image_id'], how = 'inner')
inner_Train_df.head()

,question_type,multiple_choice_answer,answers,image_id,answer_type,question_id,question,path
0,what is this,net,"[{'answer': 'net', 'answer_confidence': 'maybe...",458752,other,458752000,What is this photo taken looking through?,/content/train2014/COCO_train2014_000000458752...
1,what,pitcher,"[{'answer': 'pitcher', 'answer_confidence': 'y...",458752,other,458752001,What position is this man playing?,/content/train2014/COCO_train2014_000000458752...
2,what color is the,orange,"[{'answer': 'orange', 'answer_confidence': 'ye...",458752,other,458752002,What color is the players shirt?,/content/train2014/COCO_train2014_000000458752...
3,is this,yes,"[{'answer': 'yes', 'answer_confidence': 'yes',...",458752,yes/no,458752003,Is this man a professional baseball player?,/content/train2014/COCO_train2014_000000458752...
4,what color is the,white,"[{'answer': 'white', 'answer_confidence': 'yes...",262146,other,262146000,What color is the snow?,/content/train2014/COCO_train2014_000000262146...


In [ ]:
inner_Train_df.drop(['question_type','answers', 'question_id', 'answer_type', 'image_id'], axis = 1, inplace=True)

In [ ]:
inner_Test_df = pd.merge(QuTest_df, test_imgs, on = ['image_id'], how = 'inner')
inner_Test_df.head()

,image_id,question,question_id,path
0,262144,Is the ball flying towards the batter?,262144000,/content/test2015/COCO_test2015_000000262144.jpg
1,262144,What sport is this?,262144001,/content/test2015/COCO_test2015_000000262144.jpg
2,262144,Can you see the ball?,262144002,/content/test2015/COCO_test2015_000000262144.jpg
3,262144,Is the pitcher wearing a hat?,262144003,/content/test2015/COCO_test2015_000000262144.jpg
4,262144,Will he catch the ball in time?,262144004,/content/test2015/COCO_test2015_000000262144.jpg


In [ ]:
inner_Test_df.drop(['question_id', 'image_id'], axis = 1, inplace=True)

In [ ]:
inner_Val_df = pd.merge(Val_df, val_imgs, on = ['image_id'], how = 'inner')
inner_Val_df.head()

,question_type,multiple_choice_answer,answers,image_id,answer_type,question_id,question,path
0,none of the above,down,"[{'answer': 'down', 'answer_confidence': 'yes'...",262148,other,262148000,Where is he looking?,/content/val2014/COCO_val2014_000000262148.jpg
1,what are the,watching,"[{'answer': 'spectating', 'answer_confidence':...",262148,other,262148001,What are the people in the background doing?,/content/val2014/COCO_val2014_000000262148.jpg
2,what is,picnic table,"[{'answer': 'table', 'answer_confidence': 'yes...",262148,other,262148002,What is he on top of?,/content/val2014/COCO_val2014_000000262148.jpg
3,what,foodiebakercom,"[{'answer': 'foodiebakercom', 'answer_confiden...",393225,other,393225000,What website copyrighted the picture?,/content/val2014/COCO_val2014_000000393225.jpg
4,is this a,no,"[{'answer': 'no', 'answer_confidence': 'yes', ...",393225,yes/no,393225001,Is this a creamy soup?,/content/val2014/COCO_val2014_000000393225.jpg


In [ ]:
inner_Val_df.drop(['question_type','answers', 'question_id', 'answer_type', 'image_id'], axis = 1, inplace=True)

In [ ]:
inner_Train_df.to_csv('Final_train.csv', index=False)

In [ ]:
inner_Test_df.to_csv('Final_test.csv', index= False)

In [ ]:
inner_Val_df.to_csv('Final_val.csv', index=False)

In [ ]:
train=pd.read_csv('Final_train.csv')
val=pd.read_csv('Final_val.csv')
test=pd.read_csv('Final_test.csv')

In [ ]:
train['answer']=train['multiple_choice_answer'].map(lambda x: "[sos] "+x+" [eos]")
train.answer

0                              [sos] net [eos]
1                          [sos] pitcher [eos]
2                           [sos] orange [eos]
3                              [sos] yes [eos]
4                            [sos] white [eos]
                          ...                 
405831                       [sos] black [eos]
405832                          [sos] no [eos]
405833                       [sos] black [eos]
405834    [sos] one is easier to type on [eos]
405835                         [sos] yes [eos]
Name: answer, Length: 405836, dtype: object

In [ ]:
val['answer']=val['multiple_choice_answer'].map(lambda x: "[sos] "+x+" [eos]")
val.answer

0                   [sos] down [eos]
1               [sos] watching [eos]
2           [sos] picnic table [eos]
3         [sos] foodiebakercom [eos]
4                     [sos] no [eos]
                     ...            
194156             [sos] green [eos]
194157           [sos] go left [eos]
194158             [sos] arrow [eos]
194159              [sos] left [eos]
194160               [sos] yes [eos]
Name: answer, Length: 194161, dtype: object

In [ ]:
vocab_set=set()
tokenizer = tfds.deprecated.text.Tokenizer()

for i in val['question']:
    vocab_set.update(tokenizer.tokenize(i))
for i in train['question']:
    vocab_set.update(tokenizer.tokenize(i))
for i in test['question']:
    vocab_set.update(tokenizer.tokenize(i))


for i in val['answer']:
    vocab_set.update(tokenizer.tokenize(i))

for i in train['answer']:
    vocab_set.update(tokenizer.tokenize(i))

In [ ]:
encoder=tfds.deprecated.text.TokenTextEncoder(vocab_set)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
encoder.save_to_file('/content/drive/MyDrive/VQA_model_weights_v2/tmp2')

In [ ]:
BATCH_SIZE = 32  # Batch size for training.
EPOCHS = 10  # Number of epochs to train for.
NUM_SAMPLES = 512  # Number of samples to train on.

EMBEDDING_SIZE = 512
FF_EXPANSION = 1024
HEADS = 2

VOCAB_SIZE = len(vocab_set)

#VOCAB_SIZE =100
SEQUENCE_LENGTH = 12

In [ ]:
encoder.encode(train['question'][10])

[6052, 1426, 8182, 6703]

In [ ]:
tf.keras.utils.pad_sequences(np.array(encoder.encode(train['multiple_choice_answer'][0])).reshape(1,-1),
                              padding='post',maxlen=20)[0][0]

19655

In [ ]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs,encoder, batch_size=32, dim=(100,100), n_channels=3,
                 n_classes=10, shuffle=True,train=True,qu_maxlen=30,ans_maxlen=20):
        'Initialization'
        self.dim = dim
        self.encoder=encoder
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.train=train
        self.qu_maxlen=qu_maxlen

        self.ans_maxlen=ans_maxlen
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        #print(self.indexes[index*self.batch_size:(index+1)*self.batch_size])
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Find list of IDs
        #print(indexes)
        list_IDs_temp = self.list_IDs.iloc[indexes]
        #print(list_IDs_temp.head())

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        return X,y



    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    def format_dataset(self,inp, out):
        print(out.shape)
        return ({"encoder_inputs": inp[1],"CNN_inputs": inp[0], "decoder_inputs": out[:-1],}, out[ 1:])

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X_img = np.empty((self.batch_size, *self.dim, self.n_channels))
        X_q = np.empty((self.batch_size,self.qu_maxlen), dtype=int)

        y = np.empty((self.batch_size,self.ans_maxlen+1), dtype=int)

        # Generate data
        #print(list_IDs_temp.shape)
        for i, ID in enumerate(list_IDs_temp.iterrows()):
            # Store sample
            X_img[i] =  cv.resize(cv.imread(ID[1]['path'],), (self.dim[0], self.dim[1])).astype('float32') / 255.

            X_q[i]   =  tf.keras.utils.pad_sequences(np.array(self.encoder.encode(ID[1]['question'])).reshape(1,-1),
                              padding='post',maxlen=self.qu_maxlen)
            if self.train:
                y[i]   =  tf.keras.utils.pad_sequences(np.array(self.encoder.encode(ID[1]['answer'])).reshape(1,-1),
                              padding='post',maxlen=self.ans_maxlen+1)

        """print(np.count_nonzero(np.isnan(X_img)),
              np.count_nonzero(np.isnan(X_q)),
              np.count_nonzero(np.isnan(y)))"""
        #print(y[:,1:].shape)
        return (X_img,X_q,y[:,:-1]), y[:,1:]

# Model

In [ ]:
def positional_encoding(length, depth):
  """
  Generates a matrix of position encodings for an input sequence.

  Args:
      length: An integer representing the length of the input sequence.
      depth: An integer representing the dimensionality of the encoding.

  Returns:
      A `tf.Tensor` of shape `(length, depth)` representing the position encoding matrix.
  """
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1)

  return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
class PositionalEmbedding(tf.keras.layers.Layer):
  """
  This layer combines the input embedding with a positional encoding that helps the Transformer to understand
  the relative position of the tokens in a sequence. It takes an input sequence of tokens and converts it to
  a sequence of embedding vectors, then adds positional information to it.

  Attributes:
      vocab_size (int): The size of the vocabulary, i.e., the number of unique tokens in the input sequence.
      d_model (int): The number of dimensions in the embedding vector.

  Methods:
      compute_mask(*args, **kwargs): This method computes the mask to be applied to the embeddings.
      call(x): This method performs the computation for the layer.

  """
  def __init__(self, vocab_size, d_model,maxlen):
    """
    Initializes the PositionalEmbedding layer.

    Args:
        vocab_size (int): The size of the vocabulary, i.e., the number of unique tokens in the input sequence.
        d_model (int): The number of dimensions in the embedding vector.
    """
    super().__init__()
    self.d_model = d_model
    self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True)
    self.pos_encoding = positional_encoding(length=2048, depth=d_model)

  def compute_mask(self, *args, **kwargs):
    """
    Computes the mask to be applied to the embeddings.

    Args:
        *args: Variable length argument list.
        **kwargs: Arbitrary keyword arguments.

    Returns:
        Mask to be applied to the embeddings.
    """
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    """
    Computes the output of the layer.

    Args:
        x (tf.Tensor): Input sequence of tokens.

    Returns:
        The output sequence of embedding vectors with added positional information.
    """
    length = tf.shape(x)[1]
    x = self.embedding(x)
    # This factor sets the relative scale of the embedding and positonal_encoding.
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x

In [ ]:
class BaseAttention(tf.keras.layers.Layer):
  """
  Base Attention layer class that contains a MultiHeadAttention, LayerNormalization and Add layer.

  Attributes:
  -----------
  kwargs: dict
      keyword arguments that will be passed to the MultiHeadAttention layer during initialization.

  Methods:
  --------
  call(inputs, mask=None, training=None):
      Performs a forward pass on the input and returns the output.

  """
  def __init__(self, **kwargs):
    """
    Initializes a new instance of the BaseAttention layer class.

    Parameters:
    -----------
    kwargs: dict
        keyword arguments that will be passed to the MultiHeadAttention layer during initialization.
    """
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.ad_layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

In [ ]:
class CrossAttention(BaseAttention):
  """
  A class that implements cross-attention mechanism by inheriting from BaseAttention class.
  Cross-attention is used to process two different sequences and attends to the context sequence while processing the query sequence.
  Inherits:
      BaseAttention: A base class that defines the MultiHeadAttention layer, LayerNormalization, and Add operation.
  Args:
      **kwargs: Arguments to pass to the MultiHeadAttention layer.
  """
  def call(self, x, context,cnn):
    """
    The call function that performs the cross-attention operation.

    Args:
        x: The query sequence tensor, shape=(batch_size, seq_len, embedding_dim)
        context: The context sequence tensor, shape=(batch_size, seq_len, embedding_dim)

    Returns:
        The attended output tensor, shape=(batch_size, seq_len, embedding_dim)
    """
    context=self.ad_layernorm(self.add([context, cnn]))
    attn_output, attn_scores = self.mha(
        query=x,
        key=context,
        value=context,
        return_attention_scores=True)

    # Cache the attention scores for plotting later.
    self.last_attn_scores = attn_scores

    x = self.add([x, attn_output])
    x = self.layernorm(x)

    return x

In [ ]:
class GlobalSelfAttention(BaseAttention):
  def call(self, x):
    """
    Apply the global self-attention mechanism to the input sequence.

    Args:
        x: A tensor of shape `(batch_size, seq_len, embedding_dim)`
        representing the input sequence.

    Returns:
        A tensor of the same shape as the input, representing the sequence
        after being transformed by the self-attention mechanism.
    """
    attn_output = self.mha(
        query=x,
        value=x,
        key=x)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

In [ ]:
class CausalSelfAttention(BaseAttention):
  """
  Call self attention on the input sequence, ensuring that each position in the
  output depends only on previous positions (i.e. a causal model).

  Args:
      x: Input sequence tensor of shape `(batch_size, seq_len, embed_dim)`.

  Returns:
      Output sequence tensor of the same shape as the input, after self-attention
      and residual connection with layer normalization applied.
  """
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x,
        use_causal_mask = True)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

In [ ]:
class FeedForward(tf.keras.layers.Layer):
  """
  Implements the feedforward sublayer of the transformer block.

  Parameters:
  -----------
  d_model: int
      The number of expected features in the input and output.
  dff: int
      The number of neurons in the first Dense layer.
  dropout_rate: float, optional (default=0.1)
      The dropout rate to use.

  Attributes:
  -----------
  seq: tf.keras.Sequential
      The sequential model that applies the two Dense layers and Dropout.
  add: tf.keras.layers.Add
      The addition layer that adds the residual connection.
  layer_norm: tf.keras.layers.LayerNormalization
      The normalization layer applied to the output.

  Methods:
  --------
  call(x):
      Computes the feedforward sublayer on the input tensor x and returns the output.

  """
  def __init__(self, d_model, dff, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    """
    Passes the input tensor `x` through a feedforward network consisting of two
    dense layers with `dff` hidden units and a `relu` activation function.
    A `dropout_rate` is applied after the first dense layer to prevent overfitting.
    The output of the feedforward network is added to the original input `x` via the
    `Add()` layer. Finally, the output is normalized using the `LayerNormalization()` layer.

    Args:
        x (tf.Tensor): Input tensor with shape `(batch_size, seq_len, d_model)`.

    Returns:
        tf.Tensor: Output tensor with shape `(batch_size, seq_len, d_model)`.
    """
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x)
    return x


In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
  """
  A single layer in the transformer encoder stack.

  Args:
    d_model (int): The dimensionality of the input and output sequences.
    num_heads (int): The number of attention heads to be used in the self-attention sub-layer.
    dff (int): The number of hidden units in the feedforward sub-layer.
    dropout_rate (float): The dropout rate to be applied after the self-attention sub-layer.

  Attributes:
    self_attention (GlobalSelfAttention): A self-attention layer.
    ffn (FeedForward): A feedforward neural network layer.
  """
  def __init__(self,*, d_model, num_heads, dff, dropout_rate=0.1):
    super().__init__()

    self.self_attention = GlobalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x):
    """
    Applies the forward pass of the encoder layer.

    Args:
      x (tf.Tensor): The input sequence tensor.

    Returns:
      tf.Tensor: The output sequence tensor.
    """
    x = self.self_attention(x)
    x = self.ffn(x)
    return x

In [ ]:
class Encoder(tf.keras.layers.Layer):
  """
  A custom Keras layer that implements the encoder of a transformer-based
  neural network architecture for natural language processing tasks such
  as language translation or text classification.

  Args:
    num_layers (int): The number of layers in the encoder.
    d_model (int): The dimensionality of the output space.
    num_heads (int): The number of attention heads in the multi-head
      self-attention mechanism.
    dff (int): The dimensionality of the fully connected feedforward
      network.
    vocab_size (int): The size of the vocabulary of the input language.
    dropout_rate (float): The dropout rate to use for regularization.

  Attributes:
    d_model (int): The dimensionality of the output space.
    num_layers (int): The number of layers in the encoder.
    pos_embedding (PositionalEmbedding): The layer that learns the position
      embeddings for each token in the input sequence.
    enc_layers (list): A list of `EncoderLayer` instances, one for each
      layer in the encoder architecture.
    dropout (Dropout): The dropout layer for regularization.

  Methods:
    call(x): The forward pass of the encoder layer.

  Returns:
    The output tensor of the encoder layer, which has shape
    `(batch_size, seq_len, d_model)`.
  """
  def __init__(self, *, num_layers, d_model, num_heads,
               dff, vocab_size, dropout_rate=0.1,maxlen):
    super().__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(
        vocab_size=vocab_size, d_model=d_model,maxlen=maxlen)

    self.enc_layers = [
        EncoderLayer(d_model=d_model,
                     num_heads=num_heads,
                     dff=dff,
                     dropout_rate=dropout_rate)
        for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x):
    """
    Perform forward pass of the `Encoder` layer.

    Args:
    x: tensor of shape (batch_size, sequence_length) representing the input token IDs sequence.

    Returns:
    A tensor of shape (batch_size, sequence_length, d_model) representing the output after applying
    the self-attention and feed-forward layers to the input sequence.
    """
    # `x` is token-IDs shape: (batch, seq_len)
    x = self.pos_embedding(x)  # Shape `(batch_size, seq_len, d_model)`.

    # Add dropout.
    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x)

    return x  # Shape `(batch_size, seq_len, d_model)`.


In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
  """
  A single layer of the decoder in a transformer-based architecture.

  Args:
    d_model (int): The number of expected features in the input.
    num_heads (int): The number of attention heads.
    dff (int): The dimensionality of the feedforward network.
    dropout_rate (float): The dropout rate to be applied.

  Attributes:
    causal_self_attention: An instance of the `CausalSelfAttention` layer.
    cross_attention: An instance of the `CrossAttention` layer.
    ffn: An instance of the `FeedForward` layer.
    last_attn_scores: A tensor containing the last attention scores.

  """
  def __init__(self,
               *,
               d_model,
               num_heads,
               dff,
               dropout_rate=0.1):
    super(DecoderLayer, self).__init__()

    self.causal_self_attention = CausalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.cross_attention = CrossAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x, context,cnn):
    """
    Forward pass of the `DecoderLayer`.

    Args:
      x (tf.Tensor): The input tensor of shape
      `(batch_size, target_seq_len, d_model)`.
      context (tf.Tensor): The context tensor of shape
      `(batch_size, input_seq_len, d_model)`.

    Returns:
      The output tensor of the `DecoderLayer` of shape
      `(batch_size, target_seq_len, d_model)`.

    """
    x = self.causal_self_attention(x=x)
    x = self.cross_attention(x=x, context=context,cnn=cnn)

    # Cache the last attention scores for plotting later
    self.last_attn_scores = self.cross_attention.last_attn_scores

    x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
    return x

In [ ]:
class Decoder(tf.keras.layers.Layer):
  """A decoder model for sequence to sequence learning.

  This class implements a decoder layer for a transformer-based model used for sequence to sequence learning tasks. The decoder layer takes input embeddings, positional encodings, and attention masks as input, and returns the output of the decoder layer after applying a multi-head self-attention mechanism, followed by a cross-attention mechanism with the output from the encoder layers, and then applying a feed-forward neural network.

  Attributes:
    d_model (int): The number of output dimensions for each layer.
    num_layers (int): The number of layers in the decoder.
    pos_embedding (PositionalEmbedding): The positional embedding layer.
    dropout (Dropout): A dropout layer.
    dec_layers (list): A list of DecoderLayer objects.
    last_attn_scores (ndarray): The attention scores from the last decoder layer.

  Methods:
    call(x, context): Implements the forward pass for the decoder layer.
      Args:
        x (ndarray): A tensor of shape (batch_size, target_seq_len), representing the input token IDs.
        context (ndarray): A tensor of shape (batch_size, input_seq_len, d_model), representing the output from the encoder layers.
      Returns:
        ndarray: A tensor of shape (batch_size, target_seq_len, d_model), representing the output from the decoder layers.
  """
  def __init__(self, *, num_layers, d_model, num_heads, dff, vocab_size,
               dropout_rate=0.1,maxlen):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(vocab_size=vocab_size,
                                             d_model=d_model,
                                             maxlen=maxlen)
    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.dec_layers = [
        DecoderLayer(d_model=d_model, num_heads=num_heads,
                     dff=dff, dropout_rate=dropout_rate)
        for _ in range(num_layers)]

    self.last_attn_scores = None

  def call(self, x, context,cnn):
    """
    Implements the forward pass for the decoder layer.

    Args:
      x (ndarray): A tensor of shape (batch_size, target_seq_len), representing the input token IDs.
      context (ndarray): A tensor of shape (batch_size, input_seq_len, d_model), representing the output from the encoder layers.

    Returns:
      ndarray: A tensor of shape (batch_size, target_seq_len, d_model), representing the output from the decoder layers.
    """
    # `x` is token-IDs shape (batch, target_seq_len)
    x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)

    x = self.dropout(x)

    for i in range(self.num_layers):
      x  = self.dec_layers[i](x, context,cnn)

    self.last_attn_scores = self.dec_layers[-1].last_attn_scores

    # The shape of x is (batch_size, target_seq_len, d_model).
    return x

# training

In [ ]:

## Encoder
encoder_inputs_img = keras.Input(shape=(128,128,3), name="CNN_inputs")
encoder_inputs_txt = keras.Input(shape=(SEQUENCE_LENGTH), dtype="int64", name="encoder_inputs")


encoder_outputs = Encoder(d_model=EMBEDDING_SIZE,
                          dff=FF_EXPANSION,
                          num_heads=HEADS,
                          vocab_size=VOCAB_SIZE,
                          num_layers=1,
                          maxlen=SEQUENCE_LENGTH)(encoder_inputs_txt)


cnn=tf.keras.applications.InceptionV3(input_shape=(128,128,3),

                                                         include_top=False,
                                                        weights='imagenet',
                                                        input_tensor=encoder_inputs_img)(encoder_inputs_img)
cnn=layers.Flatten()(cnn)
cnn=layers.Dense(FF_EXPANSION,activation='relu')(cnn)
cnn=layers.RepeatVector(SEQUENCE_LENGTH)(cnn)
cnn=layers.Dense(EMBEDDING_SIZE,activation='relu')(cnn)
cnn=layers.BatchNormalization()(cnn)



## Decoder
decoder_inputs = keras.Input(shape=(SEQUENCE_LENGTH), dtype="int64", name="decoder_inputs")



#x=layers.add(cnn,encoder_outputs)
x = Decoder(d_model=EMBEDDING_SIZE,
           dff=FF_EXPANSION,
           num_heads=HEADS,
           vocab_size=VOCAB_SIZE,
           num_layers=1,
           maxlen=SEQUENCE_LENGTH)(decoder_inputs,encoder_outputs,cnn)

decoder_outputs = layers.Dense(VOCAB_SIZE,activation='softmax')(x) # 15000 token
#decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

## Transformer Model
#decoder_outputs = decoder([decoder_inputs, encoder_outputs])

transformer = keras.Model(
    [encoder_inputs_img,encoder_inputs_txt,decoder_inputs], decoder_outputs, name="transformer"
)

In [ ]:
transformer.summary()

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 CNN_inputs (InputLayer)     [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 inception_v3 (Functional)   (None, 2, 2, 2048)           2180278   ['CNN_inputs[0][0]']          
                                                          4                                       
                                                                                                  
 flatten_2 (Flatten)         (None, 8192)                 0         ['inception_v3[0][0]']        
                                                                                                  
 dense_13 (Dense)            (None, 1024)                 8389632   ['flatten_2[0][0]'] 

In [ ]:
def masked_loss(label, pred):
  """
  Calculates the masked sparse categorical cross-entropy loss between the true labels and predicted labels.

  Args:
      label: A tensor of shape (batch_size, seq_length) containing the true labels.
      pred: A tensor of shape (batch_size, seq_length, target_vocab_size) containing the predicted labels.

  Returns:
      A scalar tensor representing the masked loss value.

  """
  mask = label != 0
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=False, reduction='none')
  loss = loss_object(label, pred)

  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask

  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
  return loss

In [ ]:
def masked_accuracy(label, pred):
  """
  Calculates the masked accuracy between the true labels and predicted labels.

  Args:
      label: A tensor of shape (batch_size, seq_length) containing the true labels.
      pred: A tensor of shape (batch_size, seq_length, target_vocab_size) containing the predicted labels.

  Returns:
      A scalar tensor representing the masked accuracy value.

  """
  pred = tf.argmax(pred, axis=2)
  label = tf.cast(label, pred.dtype)
  match = label == pred

  mask = label != 0

  match = match & mask

  match = tf.cast(match, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  """
  Custom learning rate schedule that implements the learning rate function
  described in the original Transformer paper. The learning rate is increased
  linearly for the first `warmup_steps` training steps, and then decreased
  proportionally to the inverse square root of the step number.

  Args:
    d_model (int): the dimensionality of the model.
    warmup_steps (int): the number of steps taken to increase the learning rate
      linearly. Default is 4000.

  Attributes:
    d_model (float): the dimensionality of the model as a float.
    warmup_steps (int): the number of steps taken to increase the learning rate
      linearly.

  Methods:
    __call__(step): returns the learning rate at the given step.

  Returns:
    The learning rate at the given step.
  """
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    """
    Returns the learning rate at the given step.

    Args:
      step (int): the current training step.

    Returns:
      The learning rate at the given step as a float32 tensor.
    """
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
BATCH_SIZE = 32  # Batch size for training.
EPOCHS = 10  # Number of epochs to train for.
NUM_SAMPLES = 512  # Number of samples to train on.

EMBEDDING_SIZE = 512
FF_EXPANSION = 1024
HEADS = 3

VOCAB_SIZE = len(vocab_set)

#VOCAB_SIZE =100
SEQUENCE_LENGTH = 12

In [ ]:
VOCAB_SIZE

26474

In [ ]:
train.isna().sum(),val.isna().sum()

(multiple_choice_answer    0
 question                  0
 path                      0
 answer                    0
 dtype: int64,
 multiple_choice_answer    0
 question                  0
 path                      0
 answer                    0
 dtype: int64)

In [ ]:
gen=DataGenerator(train[:100000],encoder=encoder,dim=(128,128),batch_size=BATCH_SIZE,
                  qu_maxlen=SEQUENCE_LENGTH,ans_maxlen=SEQUENCE_LENGTH)

In [ ]:
x,y=gen.__getitem__(0)
x[0].shape,x[1].shape,x[2].shape,y.shape

((32, 128, 128, 3), (32, 12), (32, 12), (32, 12))

In [ ]:
gen2=DataGenerator(val[:20000],encoder=encoder,dim=(128,128),batch_size=BATCH_SIZE,
                  qu_maxlen=SEQUENCE_LENGTH,ans_maxlen=SEQUENCE_LENGTH)

In [ ]:
x,y=gen2.__getitem__(0)
x[0].shape,x[1].shape,x[2].shape,y.shape

((32, 128, 128, 3), (32, 12), (32, 12), (32, 12))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
learning_rate = CustomSchedule(EMBEDDING_SIZE)

optimizer = tf.keras.optimizers.Adam(0.0001, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)
'''transformer.compile(
    optimizer=optimizer,
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
'''
transformer.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy])

In [ ]:
#EPOCHS = 100
checkpoint_filepath = '/content/drive/MyDrive/VQA_model_weights_v2/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_masked_accuracy',
    mode='max',
    save_best_only=True)

history =  transformer.fit(gen,
                           validation_data=gen,
                           callbacks=[model_checkpoint_callback],
                           epochs=EPOCHS)


Epoch 1/10
3125/3125 [==============================] - 1973s 611ms/step - loss: 2.3060 - masked_accuracy: 0.5922 - val_loss: 1.9387 - val_masked_accuracy: 0.6103
Epoch 2/10
3125/3125 [==============================] - 1877s 600ms/step - loss: 1.9180 - masked_accuracy: 0.6160 - val_loss: 1.7941 - val_masked_accuracy: 0.6257
Epoch 3/10
3125/3125 [==============================] - 1922s 615ms/step - loss: 1.8006 - masked_accuracy: 0.6293 - val_loss: 1.6835 - val_masked_accuracy: 0.6449
Epoch 4/10
3125/3125 [==============================] - 1958s 626ms/step - loss: 1.7177 - masked_accuracy: 0.6423 - val_loss: 1.6093 - val_masked_accuracy: 0.6535
Epoch 5/10
3125/3125 [==============================] - 1929s 617ms/step - loss: 1.6505 - masked_accuracy: 0.6549 - val_loss: 1.5234 - val_masked_accuracy: 0.6734
Epoch 6/10
3125/3125 [==============================] - 1880s 601ms/step - loss: 1.5909 - masked_accuracy: 0.6672 - val_loss: 1.4670 - val_masked_accuracy: 0.6700
Epoch 7/10
3125/3125 [